<a href="https://colab.research.google.com/github/likuoping136/stable-diffusion-webui-colab/blob/main/install_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content
from google.colab import drive
drive.mount('/content/drive')

%env PYTHONDONTWRITEBYTECODE=1

# 导入必要的模块用于检查路径存在性
import os

# 检查目标目录是否存在，如果不存在则执行克隆
target_dir = '/content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui'
if not os.path.exists(target_dir):
    !git clone https://github.com/likuoping136/stable-diffusion-webui.git /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui
    print("Stable Diffusion WebUI 克隆完成。")
else:
    print("Stable Diffusion WebUI 目录已存在，跳过克隆操作。")

# 检查目标文件是否存在，如果不存在则下载
file_path = '/content/drive/MyDrive/stable-diffusion-webui-colab/libtcmalloc_minimal.so.4'
if not os.path.exists(file_path):
    !wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/drive/MyDrive/stable-diffusion-webui-colab/libtcmalloc_minimal.so.4
    print("libtcmalloc_minimal.so.4 下载完成。")
else:
    print("libtcmalloc_minimal.so.4 文件已存在，跳过下载操作。")

!apt -y install -qq unionfs-fuse aria2

# 1. 检查并下载超分辨率模型 4x-UltraSharp.pth
esrgan_model_path = '/content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/models/ESRGAN/4x-UltraSharp.pth'
if not os.path.isfile(esrgan_model_path):
    print("下载超分辨率模型: 4x-UltraSharp.pth")
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/models/ESRGAN -o 4x-UltraSharp.pth
else:
    print("超分辨率模型已存在，跳过下载。")

# 2. 检查并下载 ControlNet 模型文件（以 canny 和 lineart 为例）
controlnet_model_dir = '/content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/extensions/sd-webui-controlnet/models'

# 定义一个列表，包含要下载的模型文件信息（URL 和 输出文件名）
controlnet_models = [
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny_fp16.safetensors", "control_v11p_sd15_canny_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart_fp16.safetensors", "control_v11p_sd15_lineart_fp16.safetensors"),
    # ... 将您代码中所有的 ControlNet 模型和配置文件都按此格式添加到这个列表里
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1e_sd15_tile_fp16.yaml", "control_v11f1e_sd15_tile_fp16.yaml")
]

for model_url, output_file in controlnet_models:
    file_path = os.path.join(controlnet_model_dir, output_file)
    if not os.path.isfile(file_path):
        print(f"下载 ControlNet 模型: {output_file}")
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_url} -d {controlnet_model_dir} -o {output_file}
    else:
        print(f"ControlNet 模型 {output_file} 已存在，跳过下载。")

# 3. 检查并下载基础模型 v1-5-pruned-emaonly.ckpt
sd_model_path = '/content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/models/Stable-diffusion/v1-5-pruned-emaonly.ckpt'
if not os.path.isfile(sd_model_path):
    print("下载基础模型: v1-5-pruned-emaonly.ckpt")
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd15/resolve/main/v1-5-pruned-emaonly.ckpt -d /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/models/Stable-diffusion -o v1-5-pruned-emaonly.ckpt
else:
    print("基础模型已存在，跳过下载。")

if not os.path.exists(f"/content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/models/CLIP"):
  os.mkdir(f"/content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/models/CLIP")
  print("下载CLIP")
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://openaipublic.azureedge.net/clip/models/b8cca3fd41ae0c99ba7e8951adf17d267cdb84cd88be6f7c2e0eca1737a03836/ViT-L-14.pt -d /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/models/CLIP -o ViT-L-14.pt
else:
  print("基础模型已存在，跳过下载。")
print("模型下载检查完毕。")
# 4. 创建目录（使用 -p 参数，如果目录已存在则不会报错）
!mkdir -p /content/drive/MyDrive/stable-diffusion-webui-colab/models
!mkdir -p /content/drive/MyDrive/stable-diffusion-webui-colab/fused-models
!mkdir -p /content/drive/MyDrive/stable-diffusion-webui-colab/fused-lora
!mkdir -p /content/drive/MyDrive/stable-diffusion-webui-colab/lora

print("目录创建完毕。")

# 5. 合并路径（这部分通常不需要判断，因为它是动态挂载的）
!unionfs-fuse /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/models/Stable-diffusion=RW:/content/drive/MyDrive/stable-diffusion-webui-colab/models=RW /content/drive/MyDrive/stable-diffusion-webui-colab/fused-models
!unionfs-fuse /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora=RW:/content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/models/Lora=RW:/content/drive/MyDrive/stable-diffusion-webui-colab/lora=RW /content/drive/MyDrive/stable-diffusion-webui-colab/fused-lora

print("混合模型路径完毕")


%env LD_PRELOAD=/content/drive/MyDrive/stable-diffusion-webui-colab/libtcmalloc_minimal.so.4
%cd /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui
#!python launch.py --skip-torch-cuda-test --nostart
print("安装流程完成。")
